# Generic least squares function

This is a topical tutorial. If you are new to iminuit, you should go through the basic tutorial first. 

We have seen in the basic tutorial how to make a least-squares function with an explicit signature that iminuit could read to find the parameter names automatically. Part of the structure of a least-squares function is always the same. What changes is the model that predicts the y-values and its parameters.

Here we show how to make a generic [weighted least-squares](https://en.wikipedia.org/wiki/Weighted_least_squares) class that works with iminuit.

Note: cost functions for least-squares and maximum likelihood fits can be imported from `iminuit.cost` 

In [1]:
import numpy as np
from iminuit import Minuit
from iminuit.util import describe, make_func_code
import traceback

In [2]:
class LeastSquares:
    """
    Generic least-squares cost function with error.
    """

    def __init__(self, model, x, y, err):
        self.model = model  # model predicts y for given x
        self.x = np.asarray(x)
        self.y = np.asarray(y)
        self.err = np.asarray(err)

    def __call__(self, *par):  # we accept a variable number of model parameters
        ym = self.model(self.x, *par)
        return np.sum((self.y - ym) ** 2 / self.err ** 2)

    # NOTE: instead of the chi2, it is possible to use a custom cost function
    # def __call__(self, *par):
    #     ym = self.model(self.x, *par)
    #     z = (data_y - ym) / data_yerr**2
    #     return np.sum(z**2)

Let's try it out with iminuit.

In [3]:
def line(x, a, b):  # simple straight line model with explicit parameters
    return a + b * x

x_data = [1, 2, 3, 4, 5]
y_data = [2, 4, 6, 8, 10]
y_err = np.sqrt(y_data)

lsq = LeastSquares(line, x_data, y_data, y_err)

# this fails
try:
    m = Minuit(lsq, errordef=Minuit.LEAST_SQUARES, pedantic=False)
except:
    traceback.print_exc()

Traceback (most recent call last):
  File "<ipython-input-3-1e68a8c78a98>", line 12, in <module>
    m = Minuit(lsq, errordef=Minuit.LEAST_SQUARES, pedantic=False)
  File "src/iminuit/_libiminuit.pyx", line 614, in iminuit._libiminuit.Minuit.__init__
  File "/Users/hdembinski/Extern/iminuit/src/iminuit/util.py", line 442, in describe
    raise TypeError("Unable to obtain function signature")
TypeError: Unable to obtain function signature


What happened? iminuit uses introspection to detect the parameter names and the number of parameters. It uses the  `describe` utility for that, but it fails, since the generic method signature `LeastSquares.__call__(self, *par)`, does not reveal the number and names of the parameters.

The information could be extracted from the model signature, but iminuit knows nothing about the signature of `line(x, a, b)` here. We can fix this by generating a function signature for the `LeastSquares` class from the signature of the model.

In [4]:
# get the args from line and strip 'x'
describe(line)[1:]

['a', 'b']

In [5]:
# now inject that into the lsq object with the make_func_code tool
lsq.func_code = make_func_code(describe(line)[1:])

# now we get the right answer
describe(lsq)

['a', 'b']

We can put this code into the init function of our generic least-squares class to obtain a generic least-squares class which works with iminuit.

In [6]:
class BetterLeastSquares(LeastSquares):
    def __init__(self, model, x, y, err):
        super().__init__(model, x, y, err)
        self.func_code = make_func_code(describe(model)[1:])

In [7]:
lsq = BetterLeastSquares(line, x_data, y_data, y_err)

In [8]:
m = Minuit(lsq, pedantic=False)
m.migrad()

┌──────────────────────────────────┬──────────────────────────────────────┐
│ FCN = 4.83e-26                   │         Nfcn = 30 (30 total)         │
│ EDM = 4.83e-26 (Goal: 0.0002)    │                                      │
├───────────────┬──────────────────┼──────────────────────────────────────┤
│ Valid Minimum │ Valid Parameters │        No Parameters at limit        │
├───────────────┴──────────────────┼──────────────────────────────────────┤
│ Below EDM threshold (goal x 10)  │           Below call limit           │
├───────────────┬──────────────────┼───────────┬─────────────┬────────────┤
│   Hesse ok    │  Has Covariance  │ Accurate  │  Pos. def.  │ Not forced │
└───────────────┴──────────────────┴───────────┴─────────────┴────────────┘
┌───┬──────┬───────────┬───────────┬────────────┬────────────┬─────────┬─────────┬───────┐
│   │ Name │   Value   │ Hesse Err │ Minos Err- │ Minos Err+ │ Limit-  │ Limit+  │ Fixed │
├───┼──────┼───────────┼───────────┼────────────┼────────────┼─────────┼─────────┼───────┤
│ 0 │ a    │    0.0    │    1.8    │            │            │         │         │       │
│ 1 │ b    │    2.0    │    0.7    │            │            │         │         │       │
└───┴──────┴───────────┴───────────┴────────────┴────────────┴─────────┴─────────┴───────┘

It works :).